In [111]:
import sympy
import numpy as np

def tf2latc(b, a):
    # see: DOI: 10.1109/TAU.1973.1162522 Digital Lattice Ladder Filter Synthesis (Gray, Markel)
    def drop_small_coeffs(expr, var, tol):
        poly = sympy.Poly(expr, [var, 1/var])
        to_remove = [abs(i) for i in poly.coeffs() if abs(i) < tol]
        for i in to_remove:
            poly = poly.subs(i, 0)
        return poly.as_expr()

    w = sympy.symbols('w')
    M = len(b) - 1
    PM = 0
    AM = 0
    for n in range(M+1):
        PM += b[M-n]*w**(M-n)
        AM += a[M-n]*w**(M-n)
    #G = PM/AM

    k = sympy.Matrix.zeros(M, 1)
    v = sympy.Matrix.zeros(M+1, 1)

    Am = AM
    Pm = PM
    for m in range(M, 0, -1):
        #print('\n\nm', m)
        #print('Am:\t', Am)
        #print('Am coeffs', sympy.Poly(Am, w).coeffs())
        k[m-1] = sympy.Poly(Am, w).coeffs()[0]
        #print('am[m]',  k[m-1])
        z_Bm = (Am.subs(w, 1/w)*w**m).simplify()
        #print('z_Bm:\t', z_Bm)
        #print('Pm', Pm)
        v[m] = sympy.Poly(Pm, w).coeffs()[0]
        #print('v[m]', v[m])

        # next step
        Am = drop_small_coeffs((Am - k[m-1]*z_Bm).expand()/(1 - k[m-1]**2), w, 1e-10)
        Pm = drop_small_coeffs(Pm - z_Bm*v[m], w, 1e-10)

    v[0] = sympy.Poly(Pm, w).coeffs()[0]
    return (np.array(k[:], dtype=float), np.array(v[:], dtype=float))

In [46]:
def q31_to_double(q31_value, shift):
    if q31_value > 1 << 31 or q31_value < - (1 << 31):
        raise IOError("out of range")
    return q31_value / ( 1<< (31 - shift))

In [47]:
def double_to_q31(value, shift):
    return int(value * ( 1<< (31 - shift)))

In [110]:
b = [ 0.00969307, -0.0385379 ,  0.05769036, -0.0385379 ,  0.00969307]
a = [ 1.        , -3.92553715,  5.77937447, -3.78207718,  0.92824057]
k, v = tf2latc(b, a)
print('k: ', k)
print('v: ', v)

k:  [-0.99981143  0.99977812 -0.99902359  0.92824057]
v:  [ 6.14211180e-07  1.09028114e-06  2.09173736e-04 -4.87393617e-04
  9.69307000e-03]


In [91]:
print('k shift 1', [double_to_q31(ki, 0) for ki in reversed(k)])
print('v shift 1', [double_to_q31(vi, 0) for vi in reversed(v)])


k shift 1 [1993381445, -2145386814, 2147007153, -2147078688]
v shift 1 [20815709, -1046669, 449197, 2341, 1319]


In [105]:
b = [0, 1]
a = [0, 1]
k, v = tf2latc(b, a)
print('k: ', k)
print('v: ', v)

k:  [1.]
v:  [1. 1.]


In [ ]:
print('k: ', list(reversed(k)))
print('v: ', list(reversed(v)))